# __텍스트로 텍스트 검색하기__

- 튜토리얼 난이도: ★★☆☆☆
- 읽는데 걸리는 시간: 7분
- 사용 언어: [SQL](https://ko.wikipedia.org/wiki/SQL) (100%)
- 실행 파일 위치: tutorial/thanosql_search/search_text_by_text.ipynb
- 참고 문서: [Naver sentiment movie corpus v1.0](https://github.com/e9t/nsmc#naver-sentiment-movie-corpus-v10), [단어 임베딩: 어휘의 의미(LEXICAL SEMANTICS)를 인코딩하기](https://tutorials.pytorch.kr/beginner/nlp/word_embeddings_tutorial.html), [컴퓨터가 바라보는 문자](https://heung-bae-lee.github.io/2020/01/16/NLP_01/)

## 튜토리얼 소개

<div class="admonition note">
    <h4 class="admonition-title">텍스트 수치화 기술 이해하기</h4>
    <p>컴퓨터는 사람이 쓰는 언어(자연어)를 곧바로 입력으로 받을 수 없습니다. 때문에 자연어를 기계가 인식할 수 있는 수치 데이터로 변환하는 과정이 필요합니다. 자연어 처리분야에서 임베딩이란 이렇게 사람이 쓰는 자연어를 기계가 이해할 수 있는 숫자형태인 vector로 바꾼 결과 혹은 그 일련의 과정 전체를 의미합니다.</p>
</div>

자연어의 임베딩을 구하는 기법은 크게 통계적인 기법과, 인공신경망 기반 기법으로 구분되고, 세부적으로도 더 나눌 수 있습니다. ThanoSQL에서는 인공신경망 기반의 [자가학습모델(Self-Supervised Learning Model)](https://ko.wikipedia.org/wiki/%EC%9E%90%EA%B8%B0_%EC%A7%80%EB%8F%84_%ED%95%99%EC%8A%B5)을 사용하여, 텍스트를 잘 표현하는 벡터를 스스로 학습하는 방법을 제공합니다.

<div class="admonition note">
    <h4 class="admonition-title">본 튜토리얼에서는</h4>
    <p>👉 이번 튜토리얼에서는 영화 리뷰 데이터를 사용합니다. 데이터는 영화 리뷰 텍스트와, 라벨값으로 구성되어있지만, 자가학습법을 사용해 훈련하는 예제를 보여드리기 위해 라벨값은 사용하지 않습니다. 10,000개의 영화 리뷰 데이터를 학습하여, 입력 문장과 유사한 문장 찾기, 문장의 키워드 추출하기를 해보겠습니다.</p>
</div>

## __0. 데이터 세트 준비__

ThanoSQL의 쿼리 구문을 사용하기 위해서는 [ThanoSQL 워크스페이스](https://docs.thanosql.ai/ko/getting_started/how_to_use_ThanoSQL/#5-thanosql)에서 언급된 것처럼 API 토큰을 생성하고 아래의 쿼리를 실행해야 합니다.

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<발급받은_API_TOKEN>

### __데이터 세트 준비__

In [2]:
%%thanosql
GET THANOSQL DATASET nsmc_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" 쿼리 구문을 사용하여 원하는 데이터 세트를 워크스페이스에 저장합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>GET THANOSQL DATASET</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 데이터 세트가 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 데이터 세트는 새로운 데이터 세트로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY nsmc_train
OPTIONS (overwrite=True)
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_train.csv'

Success


In [4]:
%%thanosql
COPY nsmc_test
OPTIONS (overwrite=True)
FROM 'thanosql-dataset/nsmc_data/nsmc_sample_test.csv'

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>COPY</strong>" 쿼리 구문을 사용하여 데이터베이스에 저장 할 테이블명을 지정합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 <strong>COPY</strong>에 사용할 옵션을 지정합니다.
        <ul>
            <li>"overwrite": 동일 이름의 테이블이 데이터베이스 상에 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 테이블은 새로운 테이블로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

## __1. 데이터 세트 확인__

텍스트 수치화 모델을 만들기 위해 ThanoSQL 워크스페이스 데이터베이스에 저장되어 있는 __nsmc_train__ 테이블을 사용합니다. __nsmc_train__ 테이블은 NAVER Sentiment Movie Corpus 영화 리뷰 데이터 및 라벨 정보의 일부가 담겨 있는 테이블입니다. 아래의 쿼리 구문을 실행하고 테이블의 내용을 확인합니다.

In [5]:
%%thanosql
SELECT *
FROM nsmc_train
LIMIT 5

,id,document,label
0,9465891,저퀄리티. 뭐하나 장점이 없음.,0
1,9792014,10점 만점에 1점 !!! 최악이네 하아...,0
2,3795348,누가 억지눈물 잘흘리나 자기네들끼리 대결하는 작품,0
3,1129393,신도 싫고 인간도 싫다,1
4,4678610,추억의 명작만화,1


<div class="admonition note">
    <h4 class="admonition-title">데이터 테이블 이해하기</h4>
    <p><strong>nsmc_train</strong> 테이블은 아래와 같은 정보를 담고 있습니다.</p>
    <ul>
        <li>id: 데이터의 식별 번호</li>
        <li>document: 영화 리뷰 데이터</li>
        <li>label: 라벨값</li>
    </ul>
</div>

## __2. 텍스트 수치화 모델 생성__

이전 단계에서 확인한 __nsmc_train__ 테이블을 사용하여 텍스트 수치화 모델을 만듭니다. 아래의 쿼리 구문을 실행하여 <strong>nsmc_text_search_model</strong>이라는 이름의 모델을 만듭니다.  
(쿼리 실행 시 예상 소요 시간: 2 min)

In [6]:
%%thanosql
BUILD MODEL nsmc_text_search_model
USING SBERTKo
OPTIONS (
    text_col='document',
    overwrite=True
    )
AS
SELECT *
FROM nsmc_train

Success


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" 쿼리 구문을 사용하여 <strong>nsmc_text_search_model</strong>이라는 모델을 만들고 학습시킵니다.</li>
        <li>"<strong>USING</strong>" 쿼리 구문을 통해 베이스 모델로 <strong>SBERTKo</strong>를 사용할 것을 명시합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 모델 생성에 사용할 옵션을 지정합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"max_epochs": 텍스트 수치화 모델을 생성하기 위한 데이터 세트 학습 횟수 (int, optional, default: 1)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"overwrite": 동일 이름의 모델이 존재하는 경우 덮어쓰기 가능 여부 설정. True일 경우 기존 모델은 새로운 모델로 변경됨 (bool, optional, True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

다음 "__CONVERT USING__ " 쿼리 구문을 실행하여 __nsmc_test__ 텍스트들을 수치화 합니다. 수치화 된 결과는 __nsmc_test__ 테이블에 사용자가 옵션으로 지정한 이름(default: 'convert_result')의 컬럼에 저장됩니다.

In [7]:
%%thanosql
CONVERT USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    table_name='nsmc_test',
    batch_size=32,
    result_col='convert_result'
    )
AS
SELECT *
FROM nsmc_test

,id,document,label,convert_result
0,9181084,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[0.05522702, -0.0469979, -0.20812525, -0.72154..."
1,6318649,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[-0.8710889, -0.23248307, 0.11158937, -0.23894..."
2,5718332,유쾌하고 신나는 스피드 코미디.,1,"[-0.11198382, -0.8899959, 0.72262335, -0.11526..."
3,8063675,정말 따뜻했던 드라마ㅠㅠ,1,"[0.4988439, -0.09274278, 0.20862351, -0.450333..."
4,10229366,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[-0.036218442, -0.26977754, 0.52454, -0.138472..."
...,...,...,...,...
1995,5382227,30년간 본 수천편의 드라마중 최고는 이 작품,1,"[-0.26851672, -0.212299, 0.22983654, -0.550410..."
1996,4584773,빛돌을 이길 5명의 용사들의 패배,1,"[-0.6405299, -0.5031032, 0.19682176, 0.5325892..."
1997,2559484,빌머레이의 굳은 표정에 모든것이 살아있다.. 짐자무시 최고!,1,"[-0.43702754, 0.12926632, -0.45465574, 0.44585..."
1998,9867081,이런영화에 투자하지 않는것이 진정한 한국영화의 발전을위한 초석,0,"[0.11142391, 0.48897615, 0.5562573, 0.43545908..."


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" 쿼리 구문은 <strong>nsmc_text_search_model</strong> 모델을 텍스트 수치화를 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"text_col": 데이터 테이블에서 영화 리뷰 데이터를 담은 컬럼 (str, default: 'text')</li>
            <li>"table_name": ThanoSQL 워크스페이스 데이터베이스 내에 저장될 테이블 이름. 기존에 사용한 테이블 이름으로 지정할 경우, 기존 테이블은 'convert_result' 컬럼을 추가한 테이블로 대체. 지정하지 않을 시 테이블을 저장하지 않음 (str, optional)</li>
            <li>"batch_size": 한 번의 학습에서 읽는 데이터 세트 묶음의 크기 (int, optional, default: 16)</li>
            <li>"result_col": 데이터 테이블에서 수치화된 결과를 담을 컬럼 이름 (str, optional, default: 'convert_result')</li>
        </ul>
        </li>
    </ul>
</div>

## __3. 텍스트 수치화 모델을 사용하여 유사한 문서 검색__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서를 검색합니다.

In [8]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='영화 내용이 불만족스러웠다',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS 
    SELECT *
    FROM nsmc_test
    )

,document,label,score
0,잔잔한 영화 보기 불편할수도 있음,1,0.659124
1,영화로서는 많이 아쉬운 듯..,0,0.642425
2,너무 작위적인 스토리 전개..실망스러운 영화ㅠ,0,0.642100
3,시나리오는 한없이 후진데다가 썩은 동선이 가득한 영화,0,0.633289
4,가벼운 마음으로 웃으면서 영화를 따라가다보니 어느새 끝나있는 영화,1,0.625073
5,정말 싱겁다 영화를 다봣어도 보다만 느낌이다,0,0.623635
6,"부족한 감이 있었고, 암울했다.",0,0.618519
7,내가 생각했던 스토리와 조금 달랐지만 신선하고 독특했던 영화였다.,1,0.616638
8,"평점보고 기대해서 그런지 실망스런 영화 ,쏘쏘한 영화",0,0.610020
9,무관심했던 나를 반성케 한 영화,1,0.603797


In [9]:
%%thanosql
SELECT document, label, score
FROM (
    SEARCH TEXT
    USING nsmc_text_search_model
    OPTIONS (
        search_by='text',
        search_input='기분이 좋아지는 작품',
        emb_col='convert_result',
        result_col='score',
        top_k=10
        )
    AS
    SELECT *
    FROM nsmc_test
    )

,document,label,score
0,보고나면 기분 좋은 영화,1,0.773304
1,감동적인 영화,1,0.719860
2,너무 멋있는 영화. 환한 느낌을 준다,1,0.708152
3,몰입도 잘되고 좋은 영화,1,0.693896
4,오래됐지만 재미있는 영화,1,0.688140
5,뻔한 스토리지만 과거 추억을 생각나게 해주는 영화 재미있었어요,1,0.678594
6,멋진영화,1,0.678019
7,인상 깊은 영화다~,1,0.671168
8,오랜만에 좋은 영화를 보았다.,1,0.667201
9,추억속의 영화 ㅎ 정말 재밌게 봤습니다!,1,0.663330


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 검색 시 필요한 변수들을 정의합니다.
        <ul>
            <li>"search_by": 검색할 때 사용할 이미지|텍스트|오디오|비디오 타입 (str)</li>
            <li>"search_input": 검색할 때 사용할 입력값 (str)</li>
            <li>"emb_col": 데이터 테이블에서 수치화된 결과를 담은 컬럼 (str)</li>
            <li>"result_col": 데이터 테이블에서 검색 결과를 담을 컬럼 이름 (str, optional, default: 'search_result')</li>
            <li>"top_k": 반환할 행의 수. None을 입력할 시 데이터 테이블 전체를 반환 (int, optional, default: 1000)</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __4. 텍스트 수치화 모델을 사용하여 문서에서 키워드 추출__

이번 단계에서는 __nsmc_text_search_model__ 텍스트 수치화 모델과 테스트 테이블을 사용하여 유사한 문서에서 키워드를 추출합니다.

In [10]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT *
FROM nsmc_test
LIMIT 10 OFFSET 40

,id,document,label,convert_result,keyword
0,7928591,"일단 엠씨 역량 부족이 가장 큰 문제인듯. 시간 없다고, 재미없다고 게스트 말 끊는...",0,"[0.16518481, -0.23852126, -0.1463577, -0.12084...","{'keyword': ['안듣고 걍한마디하라고하고 끝ㅋㅋㅋ엠씨가노답임', '이야기하..."
1,8380896,방금 슬쩍 봤는데 '인삼' 처럼 생긴 '산삼' 암수가 서로 토닥이다가 주인공에게 잡...,0,"[-0.5573384, -0.28136733, -0.7538674, -0.29041...","{'keyword': ['토닥이다가 주인공에게 잡힘', '처럼 생긴 산삼', '산삼..."
2,8914833,신세계에 비교하는 거 진심 빡친다. 신성모독,1,"[0.381861, -0.20842992, -0.096296966, 1.004995...","{'keyword': ['진심 빡친다 신성모독', '신세계에 비교하는 진심', '비..."
3,9702804,킬링타임 최고다 열자채워,1,"[0.2951034, -0.38760313, -0.6527607, 0.4108700...","{'keyword': ['킬링타임 최고다 열자채워', '킬링타임 최고다', '최고다..."
4,10097178,명불허전 선동영화 10자,0,"[-0.14158297, -0.29463047, 0.31408072, -0.6441...","{'keyword': ['명불허전 선동영화 10자', '명불허전 선동영화', '명불..."
5,9245590,결말이 마음에 든다. 소설을 먼저 읽었던 터라 결말이 마음에 든다. 세상의 모든 정...,1,"[-0.118762955, -0.44482592, 0.27499488, -0.112...","{'keyword': ['읽었던 터라 결말이', '마음에 든다 세상의', '모든 정..."
6,4227156,기대했는데.. 시나리오는 ...어떻게 결말이 그렇지?..요즘 막장 드라마 영향인가..,0,"[0.38502067, -0.71749926, 0.41256008, 0.168410...","{'keyword': ['기대했는데 시나리오는 어떻게', '시나리오는 어떻게 결말이..."
7,7313784,인기연예인 몇명 출현한다고 재미있는 영화냐 잡것들아긴급조치19호가 그렇게 만들고 망...,0,"[0.33668527, -0.40426037, 0.35506713, 0.075431...","{'keyword': ['영화냐 잡것들아긴급조치19호가 그렇게', '잡것들아긴급조치..."
8,3582981,좋았음!!,1,"[0.7341071, -0.0056164996, -0.65291244, 0.5822...","{'keyword': ['좋았음'], 'score': [0.7076]}"
9,9906269,멜로와 스릴러를 이렇게 조합할수 있다니 ...,1,"[-0.30483598, -0.9592213, 0.12624064, 0.113288...","{'keyword': ['멜로와 스릴러를 이렇게', '스릴러를 이렇게 조합할수', ..."


In [11]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True
        )
    AS
    SELECT *
    FROM nsmc_test
    LIMIT 10
    )

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,"[영화인 같네요, 한번 봐야하는, 봐야하는 영화인, 같네요, ㅎㅎ]","[0.6584, 0.6418, 0.6234, 0.465, 0.4021]"
1,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,"[럽라인은작가빙의된니끼한조연과 여주는남자갖고논질나쁜여자, 권선징악은안드로메다로 럽라...","[0.8923, 0.8793, 0.7785, 0.6949, 0.6781]"
2,유쾌하고 신나는 스피드 코미디.,1,"[유쾌하고 신나는, 신나는 스피드, 스피드 코미디, 유쾌하고, 신나는]","[0.8156, 0.7876, 0.7569, 0.6816, 0.6744]"
3,정말 따뜻했던 드라마ㅠㅠ,1,"[따뜻했던 드라마ㅠㅠ, 정말 따뜻했던, 따뜻했던, 정말, 드라마ㅠㅠ]","[0.9016, 0.8892, 0.786, 0.7173, 0.584]"
4,솔직히 배우들의 연기는 나름 괜찮았다 하지만 냉정하게 영화자체만을 놓고보면 별다른 ...,0,"[솔직히 배우들의, 별다른 특색없고, 영화자체만을 놓고보면, 내세우는게 나았을듯, ...","[0.5732, 0.5308, 0.5265, 0.439, 0.4201]"
5,느와르에 멜로에 신파극까지······. 짬뽕이 아니라 꿀꿀이죽,0,"[느와르에 멜로에, 신파극까지 짬뽕이, 멜로에 신파극까지, 짬뽕이 아니라, 아니라 ...","[0.5879, 0.5807, 0.5748, 0.488, 0.4518]"
6,다시 보고 싶네요~,1,"[보고 싶네요, 다시 보고, 다시, 보고, 싶네요]","[0.798, 0.7271, 0.6249, 0.5824, 0.5289]"
7,짱이야... 진짜 극장을 나오는데 멋진 소설을 한권 읽은 듯한 기분을 느낌. 재미는...,1,"[ㅠㅠㅠ 꼭보시길, 감동까지 철철, 짱이야 진짜, 멋진 소설을, 한권 읽은]","[0.5692, 0.5395, 0.5268, 0.4572, 0.2798]"
8,유일하게 아들과 같이볼수있는 만화영화,1,"[같이볼수있는 만화영화, 아들과 같이볼수있는, 유일하게 아들과, 유일하게, 만화영화]","[0.8017, 0.7916, 0.6736, 0.5613, 0.5443]"
9,남자라면 봐라 꼭봐라 두번봐라 세번봐라 계속봐라,1,"[꼭봐라 두번봐라, 세번봐라 계속봐라, 두번봐라 세번봐라, 봐라 꼭봐라, 남자라면 봐라]","[0.8138, 0.8136, 0.8004, 0.7108, 0.5865]"


In [12]:
%%thanosql
SELECT document, label, json_array_elements(keyword -> 'keyword') AS keywords, json_array_elements(keyword -> 'score') AS score
FROM (
    SEARCH KEYWORD 
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        use_stopwords=True,
        threshold=0.5
        )
    AS
    SELECT *
    FROM nsmc_test
    LIMIT 10
    )

,document,label,keywords,score
0,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,영화인 같네요,0.6584
1,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,한번 봐야하는,0.6418
2,꼭 한번 봐야하는 영화인 것 같네요 ㅎㅎ,1,봐야하는 영화인,0.6234
3,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,럽라인은작가빙의된니끼한조연과 여주는남자갖고논질나쁜여자,0.8923
4,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악은안드로메다로 럽라인은작가빙의된니끼한조연과,0.8793
5,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,럽라인은작가빙의된니끼한조연과,0.7785
6,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,여주는남자갖고논질나쁜여자,0.6949
7,권선징악은안드로메다로~럽라인은작가빙의된니끼한조연과~여주는남자갖고논질나쁜여자,0,권선징악은안드로메다로,0.6781
8,유쾌하고 신나는 스피드 코미디.,1,유쾌하고 신나는,0.8156
9,유쾌하고 신나는 스피드 코미디.,1,신나는 스피드,0.7876


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>OPTIONS</strong>" 쿼리 구문을 통해 텍스트 수치화 시 필요한 변수들을 정의합니다.
            <ul>
                <li>"lang": 사용할 언어 (str, optional, 'ko'|'en', default: 'ko')</li>
                <li>"text_col": 텍스트가 있는 열 이름 (str, default: 'text')</li>
                <li>"ngram_range": 키워드의 최소 단어 수와 최대 단어수 예) [1, 3]. 대부분의 상황에서 키워드는 최대 단어 수에 맞춰 추출됨 (list[int, int], optional, default: [1, 2])</li>
                <li>"top_n": 추출할 키워드의 수, 유사도가 높은 순서대로 (int, optional, default: 5)</li>
                <li>"diversity": 추출될 키워드의 다양성. 높을 수록 기존에 추출된 키워드와 유사한 키워드는 다시 추출되지 않음 0 <= diversity <= 1 (float, optional, default: 0.5)</li>
                <li>"use_stopwords": 큰 의미가 없는 단어(불용어)를 제외할 지 여부 (bool, optional, True|False, default: True)</li>
                <li>"threshold": 추출할 키워드의 유사도 수치의 최소값 (float, optional, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __5. 두 방법 결합하여 사용__

In [13]:
%%thanosql
SEARCH KEYWORD
USING nsmc_text_search_model
OPTIONS (
    text_col='document',
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT document, label, score
    FROM (
        SEARCH TEXT
        USING nsmc_text_search_model
        OPTIONS (
            search_by='text',
            search_input='가볍게 볼 수 있는 코미디 영화',
            emb_col='convert_result',
            result_col='score',
            top_k=10
            )
        AS
        SELECT *
        FROM nsmc_test
        )
    )

,document,label,score,keyword
0,순수하게 보면 참으로 재밌는 영화,1,0.745830,"{'keyword': ['참으로 재밌는 영화', '순수하게 보면 참으로', '보면 ..."
1,오래됐지만 재미있는 영화,1,0.692706,"{'keyword': ['오래됐지만 재미있는 영화', '오래됐지만 재미있는', '재..."
2,보고나면 기분 좋은 영화,1,0.692283,"{'keyword': ['보고나면 기분 좋은', '기분 좋은 영화', '좋은 영화'..."
3,감동적인 영화,1,0.684076,"{'keyword': ['감동적인 영화', '감동적인', '영화'], 'score'..."
4,코미디로도 사람의 마음을 울릴수 있느 영화,1,0.678831,"{'keyword': ['사람의 마음을 울릴수', '코미디로도 사람의 마음을', '..."
5,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,0.670244,"{'keyword': ['시간가는줄 모르고 잼있게봤다', '잼있게봤다 볼만한 영화'..."
6,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,0.661960,"{'keyword': ['레전드란 말이 어울리는', '영화중 하나', '안되는 영화..."
7,가벼운 마음으로 웃으면서 영화를 따라가다보니 어느새 끝나있는 영화,1,0.648359,"{'keyword': ['어느새 끝나있는 영화', '웃으면서 영화를 따라가다보니',..."
8,간만에 본 한국영화 중 수작중의 수작,1,0.647729,"{'keyword': ['간만에 한국영화 수작중의', '한국영화 수작중의 수작', ..."
9,모든것이 흥미로운 영화였다.,1,0.640093,"{'keyword': ['모든것이 흥미로운 영화였다', '흥미로운 영화였다', '모..."


In [14]:
%%thanosql
SELECT document, label, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD
    USING nsmc_text_search_model
    OPTIONS (
        text_col='document',
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT document, label, score
        FROM (
            SEARCH TEXT
            USING nsmc_text_search_model
            OPTIONS (
                search_by='text',
                search_input='가볍게 볼 수 있는 코미디 영화',
                emb_col='convert_result',
                result_col='score',
                top_k=10
                )
            AS
            SELECT *
            FROM nsmc_test
            )
        )
    )

,document,label,keywords,score
0,순수하게 보면 참으로 재밌는 영화,1,"[참으로 재밌는 영화, 순수하게 보면 참으로, 보면 참으로 재밌는, 순수하게 보면,...","[0.8286, 0.827, 0.7848, 0.7578, 0.6822]"
1,오래됐지만 재미있는 영화,1,"[오래됐지만 재미있는 영화, 오래됐지만 재미있는, 재미있는 영화, 오래됐지만, 영화]","[1.0, 0.8834, 0.8068, 0.7868, 0.6538]"
2,보고나면 기분 좋은 영화,1,"[보고나면 기분 좋은, 기분 좋은 영화, 좋은 영화, 보고나면, 기분]","[0.9051, 0.8846, 0.8126, 0.7027, 0.6571]"
3,감동적인 영화,1,"[감동적인 영화, 감동적인, 영화]","[1.0, 0.8882, 0.6884]"
4,코미디로도 사람의 마음을 울릴수 있느 영화,1,"[사람의 마음을 울릴수, 코미디로도 사람의 마음을, 울릴수 있느 영화, 있느 영화,...","[0.7912, 0.7783, 0.7733, 0.576, 0.5047]"
5,시간가는줄 모르고 잼있게봤다 볼만한 영화,1,"[시간가는줄 모르고 잼있게봤다, 잼있게봤다 볼만한 영화, 잼있게봤다 볼만한, 모르고...","[0.9116, 0.8883, 0.8417, 0.839, 0.7046]"
6,레전드란 말이 어울리는 몇 안되는 영화중 하나,1,"[레전드란 말이 어울리는, 영화중 하나, 안되는 영화중 하나, 영화중, 말이 어울리...","[0.8035, 0.7179, 0.7057, 0.6256, 0.5073]"
7,가벼운 마음으로 웃으면서 영화를 따라가다보니 어느새 끝나있는 영화,1,"[어느새 끝나있는 영화, 웃으면서 영화를 따라가다보니, 따라가다보니 어느새 끝나있는...","[0.8077, 0.8034, 0.7471, 0.6832, 0.6745]"
8,간만에 본 한국영화 중 수작중의 수작,1,"[간만에 한국영화 수작중의, 한국영화 수작중의 수작, 간만에 한국영화, 한국영화, ...","[0.9058, 0.8619, 0.7436, 0.5837, 0.538]"
9,모든것이 흥미로운 영화였다.,1,"[모든것이 흥미로운 영화였다, 흥미로운 영화였다, 모든것이 흥미로운, 모든것이, 흥...","[0.9572, 0.7732, 0.7692, 0.6763, 0.523]"


<div class="admonition note">
    <h4 class="admonition-title">쿼리 세부 정보</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|text|audio|video]</strong>" 쿼리 구문은 검색하고자 하는 이미지|텍스트|오디오|비디오 파일을 정의합니다.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" 쿼리 구문은 키워드를 검색하기 위한 알고리즘으로 사용합니다.</li>
        <li>"<strong>USING</strong>"은 텍스트 수치화에 사용할 모델을 정의합니다.</li>
        <li>"<strong>AS</strong>" 쿼리 구문은 검색에 사용할 임베딩 테이블을 정의합니다. <strong>nsmc_test</strong> 테이블을 사용합니다.</li>
    </ul>
</div>

## __6. 튜토리얼을 마치며__

이번 튜토리얼에서는 nsmc 영화 리뷰 데이터 세트를 사용하여 텍스트 수치화와 수치화 결과를 바탕으로한 유사 텍스트 검색, 키워드 추출을 진행해 보았습니다. 이번 튜토리얼에서는 모델의 수치화 성능보다는 작동 위주의 설명으로 진행하였습니다. 나만의 텍스트 수치화 모델을 만들어 다양한 형태의 비정형 데이터 세트에 검색 기능을 추가하고 AutoML 기법을 이용한 나만의 모델을 배포할 수 있습니다.

* [나만의 데이터 업로드하기](https://docs.thanosql.ai/ko/getting_started/data_upload/)
* [나만의 데이터 테이블 생성하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [나만의 모델 업로드하기](https://docs.thanosql.ai/ko/how-to_guides/ThanoSQL_query/UPLOAD_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">나만의 서비스를 위한 모델 배포 관련 문의</h4>
    <p>ThanoSQL을 활용해 나만의 모델을 만들거나, 나의 서비스에 적용하는데 어려움이 있다면 언제든 아래로 문의주세요😊</p>
    <p>유사 텍스트 검색 모델 구축 관련 문의: <a href="mailto:contact@smartmind.team">contact@smartmind.team</a></p>
</div>